In [3]:
import numpy as np
from selenium import webdriver
from time import sleep 
import random 
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
import pandas as pd 
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from unidecode import unidecode


In [4]:
# OPEN URL
driver = uc.Chrome() 
driver.get("https://sflix.to/movie")
sleep(random.randint(5,10))


In [22]:
data_list= {
    "id":[],
    "title":[],
    "url":[],
    "vote":[],
    "year":[],
    "statuss":[]
}
df = pd.DataFrame(data_list)

In [24]:
url = "https://sflix.to/movie?page="

for i in range(901,1297):
    driver.get(url+str(i))
    soup = BeautifulSoup(str(driver.page_source), 'html.parser')
    elems = soup.find_all("a", class_= "film-poster-ahref")
    
    id = []
    url_film = []
    for elem in elems:
        url_film.append("https://sflix.to/" +elem["href"])
        id.append(elem["href"].split("-")[-1])

    elems_title = soup.find_all("h2",class_="film-name")
    # for elem in elems_title:
    #     title = elem.text
    #     print(elem.text)
    title = [elem.text for elem in elems_title]

    elems_info = soup.find_all("div",class_ = "fd-infor")
    
    year = []
    vote = []
    for elem in elems_info:
        info = elem.text.split()
        year.append(info[-1])
        vote.append(info[0])
    for index in range(len(id)):
        new_rows ={
        "id":id[index],
        "title":title[index],
        "url":url_film[index],
        "vote":vote[index],
        "year":year[index],
        "statuss":0
        }
        # page 78

        df = df.append(new_rows, ignore_index= True)
    # print(f" id = {id} ** {len(id)}")
    # print(url_film," ** ",len(url_film))
    # print(title,"** ",len(title))
    # print(year,"**",len(year))
    # print(vote,"** ",len(vote))

NameError: name 'df' is not defined

In [ ]:
df.to_csv("craw_url.csv")

In [5]:
df = pd.read_csv("craw_url.csv")

In [6]:
df

,Unnamed: 0,id,title,url,vote,year,statuss
0,0,107305,Road House,https://sflix.to//movie/free-road-house-hd-107305,NaN,2024.0,0.0
1,1,107296,The Wrong Sarah,https://sflix.to//movie/free-the-wrong-sarah-h...,4.4,2021.0,0.0
2,2,107293,"Lust, Magic, and the Witches' Sabbath",https://sflix.to//movie/free-lust-magic-and-th...,6.6,2023.0,0.0
3,3,107290,Frozen Sasquatch,https://sflix.to//movie/free-frozen-sasquatch-...,2.8,2018.0,0.0
4,4,107287,Felines,https://sflix.to//movie/free-felines-hd-107287,3.9,2023.0,0.0
...,...,...,...,...,...,...,...
28763,28763,16442,Mark Felt: The Man Who Brought Down the White ...,https://sflix.to//movie/free-mark-felt-the-man...,6.4,2017.0,0.0
28764,28764,16441,Mystic Pizza,https://sflix.to//movie/free-mystic-pizza-hd-1...,6.3,1988.0,0.0
28765,28765,16439,Margin Call,https://sflix.to//movie/free-margin-call-hd-16439,7.1,2011.0,0.0
28766,28766,16438,Lupin the Third: The Castle of Cagliostro,https://sflix.to//movie/free-lupin-the-third-t...,7.6,1979.0,0.0


cào từng page 1 

In [7]:
data_list = {
    "id":[],
    "title":[],
    "released":[],
    "genre":[],
    "casts":[],
    "duration":[],
    "country":[],
    "production":[],
    "image":[]
}
df_film = pd.DataFrame(data_list)

In [10]:
# url_film = df["url"][0]
# driver.get(url_film)

for index in range(0,20000):
    try: 
        if df["statuss"][index] != 1:
            url_film = df["url"][index]
            driver.get(url_film)
            soup = BeautifulSoup(str(driver.page_source), 'html.parser')

            description = soup.find("div",class_="description")
            description = description.text.split(":")[-1].strip()
            elems = soup.find_all("div",class_="row-line")
            
            title = soup.find("h2","heading-name")
            title = title.text

            infomation = ""
            count = 0 

            
            for elem in elems:
                count += 1
            
                
                if count == 1:
                    released_base = elem.text.split(":")
                    released = released_base[-1].strip()
                if count == 2:
                    genre_base = elem.text.split(":")
                    genre = ", ".join([i.strip() for i in genre_base[-1].split(",")])
                if count == 3:
                    casts_base = elem.text.split(":")
                    casts = ", ".join([i.strip() for i in casts_base[-1].split(",")])
                if count == 4:
                    duration_base = elem.text.split(":")
                    duration = " ".join([i.strip() for i in duration_base[-1].split("\n")])
                if count == 5:
                    country_base = elem.text.split(":")
                    country = " ".join([i.strip() for i in country_base[-1].split("\n")])
                if count == 6:
                    production_base = elem.text.split(":")
                    production = ", ".join([i.strip() for i in production_base[-1].split(", ")])

            image = soup.find("img",class_= "film-poster-img")
            image = image["src"]
            id = df["id"][index]

            new_row ={
            "id":id,
            "title":title,
            "released":released,
            "genre":genre,
            "casts":casts,
            "duration":duration,
            "country":country,
            "image":image,
            "production":production
            }
            df_film = df_film.append(new_row,ignore_index = True)
            df.at[index,"statuss"] = 1
    except:
        continue
        
            


C:\Users\FPT SHOP\AppData\Local\Temp\ipykernel_3848\659174666.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_film = df_film.append(new_row,ignore_index = True)
C:\Users\FPT SHOP\AppData\Local\Temp\ipykernel_3848\659174666.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_film = df_film.append(new_row,ignore_index = True)
C:\Users\FPT SHOP\AppData\Local\Temp\ipykernel_3848\659174666.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_film = df_film.append(new_row,ignore_index = True)
C:\Users\FPT SHOP\AppData\Local\Temp\ipykernel_3848\659174666.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_film = df_film.appe

In [ ]:
df.to_csv("craw_url.csv", mode='a', header=False)